In [28]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [29]:
from syft.grid.client.client import connect
from syft.grid.client.grid_connection import GridHTTPConnection
from syft.core.node.domain.client import DomainClient
from syft.core.plan.plan_builder import PLAN_BUILDER_VM, make_plan, ROOT_CLIENT
from syft import SyModule
from syft import logger
import syft as sy
import torch as th

sy.VERBOSE = False
logger.remove()

# Settings

In [65]:
# DOMAIN_URL, NETWORK_URL = "http://localhost:5000", "http://localhost:7000"
DOMAIN_URL, NETWORK_URL = "http://localhost:5000", "http://54.73.229.34:7001"
# DOMAIN_URL, NETWORK_URL = "http://localhost:5000", "http://3.250.104.46:7001"
do_email, do_pw = "owner@openmined.org", "12345"
ds_email, ds_pw = "data_scientist@email.com", "data_scientist_pwd123"
net_email, net_pw = "network@mymail.com", "network_pw"
token = "9G9MJ06OQH"
network_whitelist = [DOMAIN_URL]

This notebook assumes a clean database state for both domain and network, if you want to delete their databases and start from scratch you can run:

In [66]:
# !rm /Users/koen/workspace/PyGrid/apps/domain/src/nodedatabase.db
# !rm /Users/koen/workspace/PyGrid/apps/network/src/nodedatabase.db

Run a `Domain` on `http://localhost:5000` and a `Network` on `http://localhost:7000` **make sure to `export MEMORY_STORE=True`** before starting the `Network`.

# Utils

In [67]:
def setup_and_connect(url, email, pw, token, node_name="My Node", domain_name="Openmined Domain"):
    def _connect(): return connect(url=url, credentials={"email": email, "password": pw})
    try:
        return _connect()
    except Exception as e:
        client = connect(url=url)
        client.setup(email=email, password=pw, node_name=node_name, domain_name=domain_name, token=token)
        return _connect()

In [68]:
def is_associated(client, network_url):
    requests = client.association_requests.all() 
    return any([x["address"] == network_url and x["accepted"] == True for x in requests])

# Setup Network

In [70]:
network_client = setup_and_connect(NETWORK_URL, net_email, net_pw, token, domain_name="Network")

In [8]:
# network_client.association_requests.all()

# Data owner: setup, connect to network, and load data

## Setup

In [26]:
do_client = setup_and_connect(DOMAIN_URL, do_email, do_pw, token)

In [27]:
do_client.users.all()

[{'id': 1,
  'email': 'owner@openmined.org',
  'private_key': '4765364ab4e9f3bcf467ad68381314f4e78f5218427065163ab6b3c7b5f198d6',
  'verify_key': 'b7c4dd5c154dffd65a6ba431997920ffab4e4d1d1cb9a134151ea058d5ff98ea',
  'role': 4,
  'groups': []}]

## Association Request

TODO, why doesnt this work

In [13]:
network_client.association_requests.all()

NameError: name 'network_client' is not defined

In [32]:
# for r in network_client.association_requests.all():
#     network_client.association_requests.delete(association_request_id=r["id"])

In [34]:
# del network_client.association_requests[1]

In [36]:
# do_client.association_requests.all()[0]

In [14]:
if not is_associated(do_client, NETWORK_URL):
    do_client.association_requests.create(name="My request", address=NETWORK_URL, sender_address=DOMAIN_URL)

### Network Accepts requests from whitelisted urls

In [15]:
for req in network_client.association_requests.all():
    if req["address"] in network_whitelist and req["accepted"] == False:
        network_client.association_requests[req["id"]].accept();

In [16]:
network_client.association_requests.all(pandas=True)

,id,date,name,address,sender_address,accepted,pending,handshake_value
0,1,2021-05-27 12:58:14.747608,My request,http://localhost:5000,http://localhost:7000,True,False,b53a6255237890f207f2d856a994cffa893b009b1ed8e4...


## Create dataset (DO)

In [17]:
tag = "#mydataset"

In [18]:
data_x, data_y = th.randn(1024,1,28,28), th.randint(10, (1024,))

In [19]:
x_ptr = data_x.send(do_client, pointable=True, tags=[f"{tag}:x"])
y_ptr = data_y.send(do_client, pointable=True, tags=[f"{tag}:y"])

## Create DS account (DO)

If we do this for multiple `Domains` we need to make sure that we dont use the same password for different domains. As this may leak access keys between domains

In [20]:
if not any([x["email"] == ds_email for x in do_client.users.all()]):
    do_client.users.create(email=ds_email, password=ds_pw)

# Data scientist: search & train

## Search

TODO: this should ideally be a separate client (with user permissions) in the future. For now this is oke as we assume that the network owner and the data scientist are from the same org.

In [12]:
query = f"{tag}:x"

In [13]:
network_client.search(query=[query], pandas=True)

UnknownPrivateException: UnknownPrivateException has been triggered.

In [14]:
%debug

> /Users/koen/workspace/PySyft/packages/syft/src/syft/logger.py(61)traceback_and_raise()
     59     if not issubclass(type(e), Exception):
     60         e = Exception(e)
---> 61     raise e
     62 
     63 

ipdb> u
> /Users/koen/workspace/PySyft/packages/syft/src/syft/core/node/common/client.py(236)send_immediate_msg_with_reply()
    234                 exception = exception_msg.exception_type(exception_msg.exception_msg)
    235                 error(str(exception))
--> 236                 traceback_and_raise(exception)
    237             else:
    238                 return response.message

ipdb> u
> /Users/koen/workspace/PySyft/packages/syft/src/syft/grid/client/client.py(169)send_immediate_msg_with_reply()
    167     ]:
    168         return super(GridClient, self).send_immediate_msg_with_reply(  # type: ignore
--> 169             msg=msg, route_index=route_index
    170         )
    171 

ipdb> d
> /Users/koen/workspace/PySyft/packages/syft/src/syft/core/node/common/clie

In [42]:
urls = network_client.search(query=[query])["match-nodes"]
url = urls[0]

## Connect to domain

In [43]:
ds_client = connect(url=url, credentials={"email": ds_email, "password": ds_pw})

## Create plan

In [44]:
class MySyModule(SyModule):
    def __init__(self, **kwargs):
        super().__init__(**kwargs)
        self.layer1 = th.nn.Linear(28*28,100)
        self.relu1 = th.nn.ReLU()
        self.layer2 = th.nn.Linear(100,10)
    
    def forward(self, x):
        x_reshaped = x.view(-1, 28 * 28)
        o1 = self.layer1(x_reshaped)
        a1 = self.relu1(o1)
        out = self.layer2(a1)
        return out

In [45]:
model = MySyModule(input_size=(32,28*28))
dummy_dl = sy.lib.python.List([(th.randn(1024,1,28,28), th.randint(10, (1024,)))])

In [46]:
remote_torch = ROOT_CLIENT.torch

In [47]:
@make_plan
def train(x=data_x, y=data_y, model=model):
    optimizer = remote_torch.optim.SGD(model.parameters(), lr=1e-1, momentum=0)
    optimizer.zero_grad()
    out = model(x=x)[0]
    loss = remote_torch.nn.functional.cross_entropy(out, y)
    loss.backward()
    optimizer.step()

    return [model]

## Run plan

### Get dataset pointer

In [48]:
# !rm /Users/koen/workspace/PyGrid/apps/domain/src/nodedatabase.db
# !rm /Users/koen/workspace/PyGrid/apps/network/src/nodedatabase.db

In [49]:
x = [t for t in ds_client.store if f"{tag}:x" in t.tags][-1]
y = [t for t in ds_client.store if f"{tag}:y" in t.tags][-1]
model = MySyModule(input_size=(32,28*28))
train_ptr = train.send(ds_client)

TODO: think about how dummy variables in the plan can be re-used. Why do permissions not work correctly without an explicit .send()?

In [50]:
out_ptr = train_ptr(x=x, y=y, model=model.send(do_client))

We can request the resulting model

In [51]:
out_ptr.request()

In [54]:
try:
    updated_model, = out_ptr.get()
except:
    print("First DO needs to accept")
else:
    raise PermissionError("Permissions are broken")

First DO needs to accept


In [55]:
do_client.requests[-1].accept()

In [56]:
updated_model, = out_ptr.get()
updated_model

MySyModule(
  (layer1): Linear(in_features=784, out_features=100, bias=True)
  (relu1): ReLU()
  (layer2): Linear(in_features=100, out_features=10, bias=True)
)

Model is updated, so orginal and updated model are not the same

In [57]:
th.equal(list(model.parameters())[0], list(updated_model.parameters())[0])

False

# Appendix (experimental)

## DataLoader

In [39]:
from syft.util import get_root_data_path
from torchvision import datasets, transforms
mnist_path = get_root_data_path()

mnist_train = datasets.MNIST(str(mnist_path), train=True, download=True,
               transform=transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.1307,), (0.3081,))]))

mnist_test = datasets.MNIST((mnist_path), train=False, 
              transform=transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.1307,), (0.3081,))]))

train_loader = th.utils.data.DataLoader(mnist_train, batch_size=1024, shuffle=True, pin_memory=True)
test_loader = th.utils.data.DataLoader(mnist_test, batch_size=1024, shuffle=True, pin_memory=True)

In [40]:
x, y = next(iter(train_loader))

In [22]:
x_name, y_name = "x.pt", "y.pt"

In [23]:
th.save(x, x_name)

In [24]:
transforms.ToTensor??

In [25]:
tag = "mydataset"

In [26]:
# train_loader.send(domain_node)

In [17]:
# dataset = RemoteDataset(path=x_name, transform=transforms.Compose([transforms.ToTensor()]))
# dataloader = RemoteDataLoader(dataset)

## Transforms

In [18]:
from torchvision import transforms

In [19]:
# transforms.ToTensor().send(domain_node)

In [55]:
th.tensor([1,2,3]).send(domain_node, tags=[tag])

In [60]:
x_ptr = [x for x in domain_node.store if tag in x.tags][0]

In [61]:
from syft.core.remote_dataloader import RemoteDataLoader
from syft.core.remote_dataloader import RemoteDataset

In [ ]:
ten = th.rand((1000, ))

In [20]:
# domain_node.tensors

In [21]:
# domain_node.datasets.create(z)

In [22]:
# %debug

In [14]:
domain_node.datasets.all(pandas=True)

""


In [77]:
x = th.tensor([1,2,3])

In [26]:
ptr = x.send(domain_node)

In [27]:
new_ptr = ptr+5

In [32]:
new_ptr.request()

In [75]:
# new_ptr.get()

In [29]:
# domain_node.datasets